In [1]:
import pandas as pd
import numpy as np

In [14]:
df = pd.read_csv('songdata.csv')
df.head(14)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...
5,ABBA,Burning My Bridges,/a/abba/burning+my+bridges_20003011.html,"Well, you hoot and you holler and you make me ..."
6,ABBA,Cassandra,/a/abba/cassandra_20002811.html,Down in the street they're all singing and sho...
7,ABBA,Chiquitita,/a/abba/chiquitita_20002978.html,"Chiquitita, tell me what's wrong \nYou're enc..."
8,ABBA,Crazy World,/a/abba/crazy+world_20003013.html,I was out with the morning sun \nCouldn't sle...
9,ABBA,Crying Over You,/a/abba/crying+over+you_20177611.html,I'm waitin' for you baby \nI'm sitting all al...


In [3]:
df.shape

(57650, 4)

In [4]:
df = df.sample(n=5000).drop('link', axis=1).reset_index(drop=True)

In [5]:
df.shape

(5000, 3)

In [6]:
df['text'] = df['text'].str.lower().replace(r'[^\w\s]','').replace(r'\n',' ', regex=True)

In [7]:
df['text'][0]

'"mommy... mommy... mommy... mommy"      later that night in her room, abigail was drifting off to sleep.   all of what she had seen today, was hard to believe.      then the screams of a child in pain, creeping deep into her brain   then the screams of a child in pain, she cannot find her mommy   and that\'s a shame      abigail was quickly on her feet   down the stairs and into the dark, abigail was following the sound   and there she was: the child      it was clearly a ghost, the ghost from the gate in the storm   "little girl, tell me who you are?"   "i am your spirit twin from years ago, i\'m little one"      that brought back memories of abigail when turning six   sneaking out of bed, like she always did      hiding in the dark, listening to the men   tell their stories from way back then   she remembered the story of jonathan      and they\'d been talking about the little one      then the screams of a child in pain, creeping deep into her brain   then the screams of a child in

In [8]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [9]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [11]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [12]:
similarity[0]

array([1.        , 0.00648559, 0.00629897, ..., 0.00416208, 0.01527165,
       0.04836588])

In [15]:
df[df['song']=='Bang'].index[0]

3

# recommedation function

In [16]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [17]:
recommendation('Bang')

['All Night Long',
 'Afternoons In Utopia',
 'Take Comfort',
 'Lonely School Days',
 'Electricity',
 'Stunt On You',
 'Truth?',
 'I Will Wait For You',
 'King And Country',
 'Then They Do',
 'Be An Angel',
 "It's Not Unusual",
 'Islands In The Stream',
 'Kiss Tomorrow Goodbye',
 'Circles',
 'Christmas In Your Arms',
 'Fantasy',
 'Bad Seed My Daddy Sowed',
 'Railroad Days',
 'Biscuits In The Oven']

In [18]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))